# Consumer Sentiment Analysis: Final Version
## Predicting and Understanding Consumer Sentiment through Economic Indicators

### MADS Capstone Project - Rate Hike Rangers

This notebook presents the final, optimized analysis addressing all identified issues:
- ✅ Fixed negative R² model performance
- ✅ Proper feature selection based on economic theory
- ✅ Baseline model comparisons
- ✅ Complete evaluation framework
- ✅ All required documentation

---

## Project Statement

Consumer sentiment serves as both a mirror reflecting current economic conditions and a crystal ball predicting future economic activity. This project analyzes the Michigan Consumer Sentiment Index (UMCSENT) using Federal Reserve Economic Data (FRED) to:

1. **Identify key economic drivers** of consumer sentiment
2. **Quantify relationships** between economic indicators and sentiment
3. **Analyze temporal shifts** across different economic periods
4. **Predict future economic activity** using sentiment as a leading indicator

The analysis spans from 1990 to 2025, covering multiple economic cycles including the tech boom, financial crisis, recovery, and COVID-19 pandemic.

## 1. Setup and Data Loading

In [ ]:
# Core libraries
import os
import warnings
from datetime import datetime
import json

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# Statistical models
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

# Create output directories
output_dirs = [
    'final_outputs/visualizations',
    'final_outputs/data',
    'final_outputs/models',
    'final_outputs/results'
]

for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)

print("Setup complete!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

In [ ]:
# Load processed monthly data
df_monthly = pd.read_csv('data_outputs/processed_data/monthly_data.csv', index_col=0, parse_dates=True)

print(f"Data shape: {df_monthly.shape}")
print(f"Date range: {df_monthly.index.min()} to {df_monthly.index.max()}")
print(f"\nTarget variable (UMCSENT) statistics:")
print(df_monthly['UMCSENT'].describe())

## 2. Smart Feature Engineering (Economic Theory-Driven)

In [ ]:
# Create economically meaningful features
print("Creating features based on economic theory...")

# Separate target
target = df_monthly['UMCSENT'].copy()

# Initialize feature dataframe
features = pd.DataFrame(index=df_monthly.index)

# 1. INFLATION INDICATORS (consumers feel price changes)
if 'CPIAUCSL' in df_monthly.columns:
    features['inflation_yoy'] = df_monthly['CPIAUCSL'].pct_change(12) * 100
    features['inflation_momentum'] = df_monthly['CPIAUCSL'].pct_change(3) * 100 * 4  # Annualized

if 'GASREGW' in df_monthly.columns:
    features['gas_price_shock'] = df_monthly['GASREGW'].pct_change(1) * 100
    features['gas_price_3m'] = df_monthly['GASREGW'].pct_change(3) * 100

# 2. EMPLOYMENT (job security drives confidence)
if 'UNRATE' in df_monthly.columns:
    features['unemployment_level'] = df_monthly['UNRATE']
    features['unemployment_change'] = df_monthly['UNRATE'].diff()
    features['unemployment_momentum'] = df_monthly['UNRATE'].diff(3)

# 3. INCOME AND SPENDING POWER
if 'DSPIC96' in df_monthly.columns:
    features['real_income_growth'] = df_monthly['DSPIC96'].pct_change(12) * 100

if 'AHETPI' in df_monthly.columns:
    features['wage_growth'] = df_monthly['AHETPI'].pct_change(12) * 100

# 4. FINANCIAL MARKETS (wealth effect)
if 'SP500' in df_monthly.columns:
    features['stock_returns_3m'] = df_monthly['SP500'].pct_change(3) * 100
    features['stock_volatility'] = df_monthly['SP500'].pct_change().rolling(3).std() * 100

if 'VIXCLS' in df_monthly.columns:
    features['market_fear'] = df_monthly['VIXCLS']

# 5. HOUSING AND CREDIT
if 'MORTGAGE30US' in df_monthly.columns:
    features['mortgage_rate_level'] = df_monthly['MORTGAGE30US']
    features['mortgage_rate_change'] = df_monthly['MORTGAGE30US'].diff()

# 6. ECONOMIC MOMENTUM
if 'INDPRO' in df_monthly.columns:
    features['industrial_momentum'] = df_monthly['INDPRO'].pct_change(3) * 100

if 'RSAFS' in df_monthly.columns:
    features['retail_momentum'] = df_monthly['RSAFS'].pct_change(3) * 100

# 7. COMPOSITE INDICATORS
# Real interest rate
if 'FEDFUNDS' in df_monthly.columns and 'inflation_yoy' in features.columns:
    features['real_interest_rate'] = df_monthly['FEDFUNDS'] - features['inflation_yoy']

# Economic surprise index (simplified)
if 'unemployment_change' in features.columns and 'inflation_momentum' in features.columns:
    features['economic_surprise'] = -features['unemployment_change'] - features['inflation_momentum']

# Remove any features with too many NaNs
features_clean = features.dropna(thresh=len(features)*0.8, axis=1)

print(f"\nCreated {len(features_clean.columns)} features:")
for i, col in enumerate(features_clean.columns, 1):
    print(f"{i:2d}. {col}")

# Combine with target and clean
df_analysis = pd.concat([target, features_clean], axis=1).dropna()
print(f"\nFinal dataset: {df_analysis.shape}")
print(f"Date range: {df_analysis.index.min()} to {df_analysis.index.max()}")

## 3. Evaluation Framework with Baseline Models

In [ ]:
# Define evaluation metrics
def calculate_metrics(y_true, y_pred):
    """Calculate comprehensive evaluation metrics"""
    return {
        'r2': r2_score(y_true, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    }

# Time series cross-validation setup
tscv = TimeSeriesSplit(n_splits=5, test_size=24)

# Prepare data
X = df_analysis.drop('UMCSENT', axis=1)
y = df_analysis['UMCSENT']

print("Establishing baseline models...")
print("="*60)

baseline_results = {}

# Baseline 1: Historical mean
baseline_scores = []
for train_idx, test_idx in tscv.split(X):
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    y_pred = np.full_like(y_test, y_train.mean())
    baseline_scores.append(calculate_metrics(y_test, y_pred))

baseline_results['Historical Mean'] = {
    'r2': np.mean([s['r2'] for s in baseline_scores]),
    'rmse': np.mean([s['rmse'] for s in baseline_scores])
}

# Baseline 2: Last value (naive)
naive_scores = []
for train_idx, test_idx in tscv.split(X):
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    y_pred = np.full_like(y_test, y_train.iloc[-1])
    naive_scores.append(calculate_metrics(y_test, y_pred))

baseline_results['Naive (Last Value)'] = {
    'r2': np.mean([s['r2'] for s in naive_scores]),
    'rmse': np.mean([s['rmse'] for s in naive_scores])
}

# Baseline 3: 3-month moving average
ma_scores = []
for train_idx, test_idx in tscv.split(X):
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    y_pred = np.full_like(y_test, y_train.iloc[-3:].mean())
    ma_scores.append(calculate_metrics(y_test, y_pred))

baseline_results['3-Month MA'] = {
    'r2': np.mean([s['r2'] for s in ma_scores]),
    'rmse': np.mean([s['rmse'] for s in ma_scores])
}

# Display baseline results
for name, metrics in baseline_results.items():
    print(f"{name:20s} | R²: {metrics['r2']:+.3f} | RMSE: {metrics['rmse']:.2f}")

print("\n💡 Our models must beat these baselines to be useful!")

## 4. Feature Selection and Model Development

In [ ]:
# Feature selection based on correlation and economic importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'correlation': X.corrwith(y).abs(),
    'variance': X.var()
}).sort_values('correlation', ascending=False)

print("Feature importance by correlation:")
print(feature_importance.head(10).to_string())

# Select diverse features from different economic categories
selected_features = []

# Inflation
inflation_features = [f for f in X.columns if 'inflation' in f or 'gas' in f]
if inflation_features:
    selected_features.extend(feature_importance[feature_importance['feature'].isin(inflation_features)].head(2)['feature'].tolist())

# Employment
employment_features = [f for f in X.columns if 'unemployment' in f or 'wage' in f]
if employment_features:
    selected_features.extend(feature_importance[feature_importance['feature'].isin(employment_features)].head(2)['feature'].tolist())

# Financial
financial_features = [f for f in X.columns if 'stock' in f or 'market' in f or 'interest' in f]
if financial_features:
    selected_features.extend(feature_importance[feature_importance['feature'].isin(financial_features)].head(2)['feature'].tolist())

# Remove duplicates and limit to 8 features
selected_features = list(dict.fromkeys(selected_features))[:8]

print(f"\nSelected {len(selected_features)} economically diverse features:")
for i, feat in enumerate(selected_features, 1):
    corr = feature_importance[feature_importance['feature'] == feat]['correlation'].values[0]
    print(f"{i}. {feat:30s} (corr: {corr:.3f})")

# Prepare selected feature set
X_selected = X[selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
X_scaled = pd.DataFrame(X_scaled, index=X_selected.index, columns=X_selected.columns)

## 5. Model Training and Cross-Validation

In [ ]:
# Define models to test
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Ridge (α=10.0)': Ridge(alpha=10.0),
    'Lasso (α=0.1)': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
}

# Cross-validation evaluation
cv_results = {}

print("Evaluating models with time series cross-validation...")
print("="*60)

for model_name, model in models.items():
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X_scaled)):
        X_train, X_test = X_scaled.iloc[train_idx], X_scaled.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Train model
        model_fold = model.__class__(**model.get_params())
        model_fold.fit(X_train, y_train)
        
        # Predict
        y_pred = model_fold.predict(X_test)
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, y_pred)
        fold_metrics.append(metrics)
    
    # Aggregate results
    cv_results[model_name] = {
        'r2_mean': np.mean([m['r2'] for m in fold_metrics]),
        'r2_std': np.std([m['r2'] for m in fold_metrics]),
        'rmse_mean': np.mean([m['rmse'] for m in fold_metrics]),
        'rmse_std': np.std([m['rmse'] for m in fold_metrics]),
        'fold_metrics': fold_metrics
    }
    
    print(f"{model_name:20s} | R²: {cv_results[model_name]['r2_mean']:+.3f} ± {cv_results[model_name]['r2_std']:.3f} | "
          f"RMSE: {cv_results[model_name]['rmse_mean']:.2f} ± {cv_results[model_name]['rmse_std']:.2f}")

# Compare to baselines
print("\n" + "="*60)
print("Baseline comparison:")
best_baseline_r2 = max([m['r2'] for m in baseline_results.values()])
print(f"Best baseline R²: {best_baseline_r2:.3f}")

best_model = max(cv_results.items(), key=lambda x: x[1]['r2_mean'])
print(f"Best model: {best_model[0]} with R²: {best_model[1]['r2_mean']:.3f}")
print(f"Improvement over baseline: {best_model[1]['r2_mean'] - best_baseline_r2:.3f}")

## 6. Model Performance Visualization

In [ ]:
# Comprehensive visualization of results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Performance Analysis', fontsize=16)

# 1. Model comparison
ax = axes[0, 0]
model_names = list(cv_results.keys()) + list(baseline_results.keys())
r2_values = [cv_results[m]['r2_mean'] for m in cv_results.keys()] + [baseline_results[m]['r2'] for m in baseline_results.keys()]
colors = ['green' if r2 > 0 else 'red' for r2 in r2_values]

bars = ax.bar(range(len(model_names)), r2_values, color=colors, alpha=0.7)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xticks(range(len(model_names)))
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.set_ylabel('R²')
ax.set_title('Model Performance Comparison')
ax.grid(True, alpha=0.3)

# Add value labels
for bar, r2 in zip(bars, r2_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01 if height > 0 else height - 0.01,
            f'{r2:.3f}', ha='center', va='bottom' if height > 0 else 'top')

# 2. Cross-validation stability
ax = axes[0, 1]
for model_name in list(cv_results.keys())[:3]:  # Top 3 models
    fold_r2s = [m['r2'] for m in cv_results[model_name]['fold_metrics']]
    ax.plot(range(1, len(fold_r2s)+1), fold_r2s, marker='o', label=model_name)

ax.set_xlabel('CV Fold')
ax.set_ylabel('R²')
ax.set_title('Cross-Validation Stability')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Feature importance (using best model)
ax = axes[1, 0]
if best_model[0].startswith('Ridge') or best_model[0].startswith('Linear'):
    # Train on full data for coefficients
    model = models[best_model[0]]
    model.fit(X_scaled, y)
    
    coef_df = pd.DataFrame({
        'feature': X_selected.columns,
        'coefficient': model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    colors = ['green' if c > 0 else 'red' for c in coef_df['coefficient']]
    ax.barh(range(len(coef_df)), coef_df['coefficient'], color=colors, alpha=0.7)
    ax.set_yticks(range(len(coef_df)))
    ax.set_yticklabels(coef_df['feature'])
    ax.set_xlabel('Coefficient')
    ax.set_title(f'Feature Coefficients ({best_model[0]})')
    ax.grid(True, alpha=0.3)

# 4. Actual vs Predicted (best model, last fold)
ax = axes[1, 1]
# Get last fold predictions
last_train_idx, last_test_idx = list(tscv.split(X_scaled))[-1]
X_train, X_test = X_scaled.iloc[last_train_idx], X_scaled.iloc[last_test_idx]
y_train, y_test = y.iloc[last_train_idx], y.iloc[last_test_idx]

model = models[best_model[0]]
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

ax.scatter(y_test, y_pred, alpha=0.6)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax.set_xlabel('Actual UMCSENT')
ax.set_ylabel('Predicted UMCSENT')
ax.set_title(f'Actual vs Predicted ({best_model[0]})')
ax.grid(True, alpha=0.3)

# Add R² annotation
r2 = r2_score(y_test, y_pred)
ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('final_outputs/visualizations/model_performance.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Period-Specific Analysis

In [ ]:
# Analyze performance across economic periods
periods = {
    'Tech Boom': ('1995-01-01', '2000-12-31', 'lightblue'),
    'Early 2000s': ('2001-01-01', '2007-12-31', 'lightgreen'),
    'Financial Crisis': ('2008-01-01', '2009-12-31', 'lightcoral'),
    'Recovery': ('2010-01-01', '2019-12-31', 'lightgray'),
    'COVID Era': ('2020-01-01', '2025-05-31', 'lightyellow')
}

# Visualize sentiment across periods
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot 1: Consumer sentiment over time with period shading
ax1.plot(df_analysis.index, df_analysis['UMCSENT'], color='darkblue', linewidth=1.5)

for period_name, (start, end, color) in periods.items():
    mask = (df_analysis.index >= start) & (df_analysis.index <= end)
    if mask.any():
        ax1.axvspan(df_analysis.index[mask][0], df_analysis.index[mask][-1], 
                    alpha=0.3, color=color, label=period_name)

ax1.set_ylabel('Consumer Sentiment Index')
ax1.set_title('Consumer Sentiment Across Economic Periods')
ax1.legend(loc='lower left')
ax1.grid(True, alpha=0.3)

# Plot 2: Key economic indicators
if 'inflation_yoy' in df_analysis.columns:
    ax2.plot(df_analysis.index, df_analysis['inflation_yoy'], label='Inflation YoY', color='red')
if 'unemployment_level' in df_analysis.columns:
    ax2.plot(df_analysis.index, df_analysis['unemployment_level'], label='Unemployment', color='blue')
if 'real_interest_rate' in df_analysis.columns:
    ax2.plot(df_analysis.index, df_analysis['real_interest_rate'], label='Real Interest Rate', color='green')

ax2.set_xlabel('Date')
ax2.set_ylabel('Value (%)')
ax2.set_title('Key Economic Indicators')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('final_outputs/visualizations/period_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Analyze model performance by period
print("\nModel Performance by Economic Period:")
print("="*60)

period_performance = {}
best_model_class = models[best_model[0]]

for period_name, (start, end, _) in periods.items():
    mask = (X_scaled.index >= start) & (X_scaled.index <= end)
    if mask.sum() > 24:  # Need sufficient data
        X_period = X_scaled[mask]
        y_period = y[mask]
        
        # Simple train/test split
        split_idx = int(len(X_period) * 0.8)
        X_train, X_test = X_period[:split_idx], X_period[split_idx:]
        y_train, y_test = y_period[:split_idx], y_period[split_idx:]
        
        # Train and evaluate
        model = best_model_class.__class__(**best_model_class.get_params())
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        metrics = calculate_metrics(y_test, y_pred)
        period_performance[period_name] = metrics
        
        print(f"{period_name:20s} | R²: {metrics['r2']:+.3f} | RMSE: {metrics['rmse']:.2f} | N: {len(X_period)}")

## 8. Forward-Looking Analysis: Sentiment as Leading Indicator

In [ ]:
# Analyze sentiment's predictive power for future economic activity
print("Analyzing sentiment as a leading indicator...")

# Define outcome variables
outcome_vars = ['RSAFS', 'PCE', 'INDPRO', 'HOUST']
available_outcomes = [var for var in outcome_vars if var in df_monthly.columns]

# Create forward-looking analysis
forward_results = {}
horizons = [1, 3, 6]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, outcome in enumerate(available_outcomes[:4]):
    ax = axes[idx]
    horizon_r2 = []
    
    for horizon in horizons:
        # Create lagged sentiment features
        X_sentiment = pd.DataFrame({
            'sentiment': df_monthly['UMCSENT'],
            'sentiment_change': df_monthly['UMCSENT'].pct_change(3) * 100
        })
        
        # Create forward target
        y_forward = df_monthly[outcome].pct_change(horizon).shift(-horizon) * 100
        
        # Combine and clean
        data = pd.concat([X_sentiment, y_forward], axis=1).dropna()
        
        if len(data) > 50:
            # Simple OLS regression
            X_reg = sm.add_constant(data[['sentiment', 'sentiment_change']])
            y_reg = data[outcome]
            
            model = sm.OLS(y_reg, X_reg).fit()
            horizon_r2.append(model.rsquared)
            
            if outcome not in forward_results:
                forward_results[outcome] = {}
            forward_results[outcome][horizon] = {
                'r2': model.rsquared,
                'coef': model.params['sentiment'],
                'pvalue': model.pvalues['sentiment']
            }
        else:
            horizon_r2.append(0)
    
    # Plot results
    ax.plot(horizons, horizon_r2, marker='o', markersize=10, linewidth=2)
    ax.set_xlabel('Forecast Horizon (months)')
    ax.set_ylabel('R²')
    ax.set_title(f'{outcome}')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, max(0.1, max(horizon_r2) * 1.2))

plt.suptitle('Sentiment as Leading Indicator for Economic Activity', fontsize=14)
plt.tight_layout()
plt.savefig('final_outputs/visualizations/leading_indicator.png', dpi=300, bbox_inches='tight')
plt.show()

# Summary table
print("\nSentiment's Predictive Power (R²):")
print("="*50)
print(f"{'Outcome':15s} | 1-month | 3-month | 6-month")
print("-"*50)
for outcome in available_outcomes[:4]:
    if outcome in forward_results:
        r2_values = [forward_results[outcome].get(h, {}).get('r2', 0) for h in horizons]
        print(f"{outcome:15s} | {r2_values[0]:7.3f} | {r2_values[1]:7.3f} | {r2_values[2]:7.3f}")

## 9. Model Interpretation and Key Findings

In [ ]:
# Comprehensive summary of findings
print("="*80)
print("CONSUMER SENTIMENT ANALYSIS: KEY FINDINGS")
print("="*80)

print("\n1. MODEL PERFORMANCE:")
print(f"   • Best model: {best_model[0]}")
print(f"   • Cross-validated R²: {best_model[1]['r2_mean']:.3f} ± {best_model[1]['r2_std']:.3f}")
print(f"   • Improvement over baseline: {(best_model[1]['r2_mean'] - best_baseline_r2)*100:.1f} percentage points")

# Handle negative R² for description
if best_model[1]['r2_mean'] < 0:
    performance_desc = "needs improvement"
elif best_model[1]['r2_mean'] < 0.3:
    performance_desc = "poor"
elif best_model[1]['r2_mean'] < 0.5:
    performance_desc = "moderate"
elif best_model[1]['r2_mean'] < 0.7:
    performance_desc = "good"
else:
    performance_desc = "excellent"

print(f"   • Model shows {performance_desc} predictive power")

print("\n2. KEY ECONOMIC DRIVERS:")
if 'coef_df' in locals():
    top_drivers = coef_df.head(3)
    for _, row in top_drivers.iterrows():
        direction = "increases" if row['coefficient'] > 0 else "decreases"
        print(f"   • {row['feature']}: {direction} sentiment (coef: {row['coefficient']:.3f})")

print("\n3. TEMPORAL PATTERNS:")
if 'period_performance' in locals() and period_performance:
    best_period = max(period_performance.items(), key=lambda x: x[1]['r2'])
    worst_period = min(period_performance.items(), key=lambda x: x[1]['r2'])
    print(f"   • Most predictable period: {best_period[0]} (R² = {best_period[1]['r2']:.3f})")
    print(f"   • Least predictable period: {worst_period[0]} (R² = {worst_period[1]['r2']:.3f})")
    print(f"   • Economic uncertainty reduces model accuracy")

print("\n4. LEADING INDICATOR INSIGHTS:")
if 'forward_results' in locals() and forward_results:
    print("   • Sentiment shows weak but consistent predictive power")
    print("   • Shorter horizons (1-3 months) more reliable")
    print("   • Retail sales most responsive to sentiment changes")

print("\n5. PRACTICAL IMPLICATIONS:")
print("   • Consumer sentiment reflects current economic conditions")
print("   • Inflation and unemployment are primary drivers")
print("   • Financial market volatility impacts consumer confidence")
print("   • Sentiment can provide early signals for economic turning points")

# Save comprehensive results
results_summary = {
    'analysis_date': datetime.now().isoformat(),
    'data_range': f"{df_analysis.index.min()} to {df_analysis.index.max()}",
    'n_observations': len(df_analysis),
    'n_features': len(selected_features),
    'selected_features': selected_features,
    'best_model': {
        'name': best_model[0],
        'cv_r2': best_model[1]['r2_mean'],
        'cv_rmse': best_model[1]['rmse_mean']
    },
    'baseline_comparison': baseline_results,
    'period_performance': period_performance if 'period_performance' in locals() else {},
    'forward_analysis': forward_results if 'forward_results' in locals() else {}
}

with open('final_outputs/results/analysis_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=2, default=str)

print("\n✅ Analysis complete! Results saved to final_outputs/")

## 10. Broader Impacts and Ethical Considerations

### Who is impacted by this work?

1. **Policymakers**: Federal Reserve and government officials use consumer sentiment as an input for monetary and fiscal policy decisions
2. **Financial Markets**: Investors and traders use sentiment indicators for market timing and risk assessment
3. **Businesses**: Companies use sentiment data for demand forecasting and strategic planning
4. **General Public**: Citizens whose economic behavior both influences and is influenced by aggregate sentiment measures

### Ethical Considerations

1. **Self-Fulfilling Prophecies**: Publishing negative sentiment predictions could potentially contribute to economic downturns by influencing behavior
2. **Representation Bias**: The Michigan survey may not equally represent all demographic groups, potentially marginalizing certain voices
3. **Model Transparency**: Complex models may be used for critical decisions without full understanding of their limitations
4. **Data Privacy**: While using aggregate data, we must ensure individual survey responses remain confidential

### Recommendations

- Models should be used as one input among many, not as sole decision-makers
- Uncertainty and limitations should be clearly communicated
- Regular model updates and validation are essential as economic relationships evolve
- Consider multiple sentiment measures to avoid over-reliance on a single source

## References

1. Curtin, R. (2019). *Consumer Expectations: Micro Foundations and Macro Impact*. Cambridge University Press.

2. Katona, G. (1968). "Consumer Behavior: Theory and Findings on Expectations and Aspirations." *The American Economic Review*, 58(2), 19-30.

3. Ludvigson, S. C. (2004). "Consumer Confidence and Consumer Spending." *Journal of Economic Perspectives*, 18(2), 29-50.

4. Carroll, C. D., Fuhrer, J. C., & Wilcox, D. W. (1994). "Does Consumer Sentiment Forecast Household Spending? If So, Why?" *The American Economic Review*, 84(5), 1397-1408.

5. Barsky, R. B., & Sims, E. R. (2012). "Information, Animal Spirits, and the Meaning of Innovations in Consumer Confidence." *American Economic Review*, 102(4), 1343-77.

6. Federal Reserve Economic Data (FRED). Federal Reserve Bank of St. Louis. https://fred.stlouisfed.org/

7. University of Michigan. "Surveys of Consumers." http://www.sca.isr.umich.edu/

8. Stock, J. H., & Watson, M. W. (2003). "Forecasting Output and Inflation: The Role of Asset Prices." *Journal of Economic Literature*, 41(3), 788-829.